In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

rm: impossible de supprimer './test_db3.db': Aucun fichier ou dossier de ce type


In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number']))
   # components_values = association_proxy(
   #     'components', 'value',
   #     creator=lambda k, v: Component(id=k, description=v['description']))
    
class Position(Base):
    """The Position table"""
    __tablename__ = 'Positions'
    id = Column(Integer, ForeignKey('Intervals.Position_id'), primary_key=True)
    upper = Column(Float(32))
    middle = Column(Float(32))
    lower = Column(Float(32))
    Position_name = Column(String(32))
    
class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id'))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    Position_id =  Column(Integer)
    Position_description = relationship('Position', collection_class=attribute_mapped_collection('id'), cascade='all, delete-orphan')
    description = Column(String(32))
    #top = Column(Position())
    #base = Column(Position(upper, middle, lower))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-18 10:27:28,827 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 10:27:28,830 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 10:27:28,834 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 10:27:28,838 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 10:27:28,842 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-18 10:27:28,844 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 10:27:28,851 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-18 10:27:28,854 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 10:27:28,861 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2020-12-18 10:27:28,863 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 10:27:28,866 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2020-12-18 10:27:28,868 INFO sqlalchemy.engine.base.Engine ()
2020-12-18

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
from striplog import Lexicon, Striplog

In [8]:
lexicon = Lexicon.default()

In [9]:
borehole_dict = {'F01':'test.las', 'F02':'test.las'} # boreholes to insert into the db

In [10]:
int_id = 0
bh_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    # print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

In [11]:
components

{0: Component({'lithology': 'siltstone', 'colour': 'grey'}),
 1: Component({'lithology': 'anhydrite'}),
 2: Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 3: Component({'lithology': 'dolomite'}),
 4: Component({'lithology': 'siltstone', 'colour': 'red'}),
 5: Component({'lithology': 'limestone'})}

In [12]:
component_dict

{Component({'lithology': 'siltstone', 'colour': 'grey'}): 0,
 Component({'lithology': 'anhydrite'}): 1,
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}): 2,
 Component({'lithology': 'dolomite'}): 3,
 Component({'lithology': 'siltstone', 'colour': 'red'}): 4,
 Component({'lithology': 'limestone'}): 5}

In [13]:
strip.components

[Component({'lithology': 'siltstone', 'colour': 'grey'}),
 Component({'lithology': 'anhydrite'}),
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 Component({'lithology': 'dolomite'}),
 Component({'lithology': 'siltstone', 'colour': 'red'}),
 Component({'lithology': 'limestone'})]

In [14]:
strip[0].components

[Component({'lithology': 'anhydrite'})]

In [15]:
boreholes

[<__main__.Borehole at 0x7f74015201d0>, <__main__.Borehole at 0x7f74014c6f60>]

In [16]:
for id in components.keys():
    print(id, components[id])

0 {'lithology': 'siltstone', 'colour': 'grey'}
1 {'lithology': 'anhydrite'}
2 {'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}
3 {'lithology': 'dolomite'}
4 {'lithology': 'siltstone', 'colour': 'red'}
5 {'lithology': 'limestone'}


In [17]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)
        #for iv in bh.intervals:
        #    print(bh.intervals[iv].components[0])
        
with session_scope() as session:
    for id in components.keys():
        new_compo = Component(id=id, description=components[id].summary()) 
        session.add(new_compo)

2020-12-18 10:38:58,042 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 10:38:58,046 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-18 10:38:58,048 INFO sqlalchemy.engine.base.Engine (('F01',), ('F02',))
2020-12-18 10:38:58,063 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, "Position_id", description) VALUES (?, ?, ?, ?, ?)
2020-12-18 10:38:58,066 INFO sqlalchemy.engine.base.Engine ((0, 'F01', 0, None, 'Anhydrite'), (1, 'F01', 1, None, 'Sandstone, grey, vf-f'), (2, 'F01', 2, None, 'Anhydrite'), (3, 'F01', 3, None, 'Dolomite'), (4, 'F01', 4, None, 'Anhydrite'), (5, 'F01', 5, None, 'Sandstone, grey, vf-f'), (6, 'F01', 6, None, 'Siltstone, red'), (7, 'F01', 7, None, 'Dolomite')  ... displaying 10 of 50 total bound parameter sets ...  (48, 'F02', 23, None, 'Limestone'), (49, 'F02', 24, None, 'Volcanic'))
2020-12-18 10:38:58,073 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-18 10:38:58,229 INF

In [42]:
boreholes[0].intervals[0]

DetachedInstanceError: Parent instance <Borehole at 0x7f74015201d0> is not bound to a Session; lazy load operation of attribute 'intervals' cannot proceed (Background on this error at: http://sqlalche.me/e/13/bhk3)

In [46]:
with session_scope() as session:
    new_position = Position(id=1, upper=-2, lower = -5, middle = -4, Position_name = "base") 
    session.add(new_position)

2020-12-18 11:03:53,273 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 11:03:53,276 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, "Position_name") VALUES (?, ?, ?, ?, ?)
2020-12-18 11:03:53,278 INFO sqlalchemy.engine.base.Engine (1, -2.0, -4.0, -5.0, 'base')
2020-12-18 11:03:53,282 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
borehole_dict

{'F01': 'test.las', 'F02': 'test.las'}

In [44]:
interval

Interval({'top': Position({'middle': 131.47647058823532, 'units': 'm'}), 'base': Position({'middle': 135.9, 'units': 'm'}), 'description': 'Volcanic', 'data': {}, 'components': [Component({})]})

In [45]:
interval.top

upper,131.47647058823532
middle,131.47647058823532
lower,131.47647058823532
